# Lesson 83 and 84

In [3]:
#from sklearn.datasets import load_boston
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np


In [4]:
# Gather Data
#boston_dataset = load_boston()
boston_dataset = fetch_openml(name='boston', version=1)
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
#features.dtypes
features['CHAS'] = features['CHAS'].astype('float64')
features['RAD'] = features['RAD'].astype('float64')
#features.dtypes

In [5]:
log_prices = np.log(boston_dataset.target).to_numpy()
# add to_numpy() method to force return a Array otherwise returns a serie
#print(type(log_prices))
target = pd.DataFrame(log_prices, columns=['PRICE'])
#print(target)

In [43]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = features.mean().to_numpy().reshape(1,11)
#property_stats = features.mean().values.reshape(1, 11)
#property_stats
property_stats = features.mean().to_frame().transpose()


In [44]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Chalenge: calculate MSE and RMSE using scikit-learn
MSE = mean_squared_error(target, fitted_vals )
RMSE = np.sqrt(MSE)
RMSE

np.float64(0.18751213519713034)

In [47]:
def get_log_estimate(nr_rooms, students_per_classroom, 
                     next_to_river=False, 
                     high_confidence=False):
    # Configure Property
    #property_stats[0][RM_IDX] = nr_rooms
    property_stats['RM'] = nr_rooms
    #property_stats[0][PTRATIO_IDX] = students_per_classroom
    property_stats['PTRATIO'] = students_per_classroom
    if next_to_river:
        #property_stats[0][CHAS_IDX] = 1
        property_stats['CHAS'] = 1
    else:
        #property_stats[0][CHAS_IDX] = 0
        property_stats['CHAS'] = 0

    # Make prediction
    #log_estimate = regr.predict(property_stats)[0][0]
    log_estimate = regr.predict(property_stats)

    # Calc range
    if high_confidence:
        # calc 2 ST. D
        upper_bound = log_estimate + 2* RMSE
        lower_bound = log_estimate - 2* RMSE
        interval = 95
    else:
        # calc 1 ST. D.
        # calc 2 ST. D
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval


In [48]:
get_log_estimate(3,10, next_to_river=False, high_confidence=True)

(array([[3.04586876]]), array([[3.42089303]]), array([[2.67084449]]), 95)

In [49]:
np.median(boston_dataset.target)

np.float64(21.2)

In [50]:
#Chalenge: write teh python code that converts the log price estimate using 1970s prices
#as well as the uppeer and the lowe bounds to today's prices. Roud the value to  the
#nearest 1000 dollars
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE /np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom= 15,
                                                   next_to_river=False,
                                                    high_confidence=False )

# convert to today's dollar
dollar_est = np.e**(log_est) * 1000 * SCALE_FACTOR
dollar_up = np.e**(upper) * 1000 * SCALE_FACTOR
dollar_low = np.e**(lower) * 1000 * SCALE_FACTOR

rounded_est = np.around(dollar_est,-3)
rounded_up = np.around(dollar_up,-3)
rounded_low = np.around(dollar_low,-3)

print(f'The estimate property value is USD {rounded_est}')
print(f'At {conf}% confidence of valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_up} at higher end')

The estimate property value is USD [[827000.]]
At 68% confidence of valuation range is
USD [[685000.]] at the lower end to USD [[997000.]] at higher end


In [55]:
def get_dollar_estimate(rm, ptratio, chas=False, range=True):
    """Estimate the price of a property in Boston.

    Keyword arguments:
    rm -- number of rooms int the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.

    """

    if rm < 1 or ptratio < 1 :
        print('That is unrealistic! Try again.')
        return
    

    log_est, upper, lower, conf = get_log_estimate(rm, ptratio,
                                                   next_to_river=chas,
                                                    high_confidence=range )

    # convert to today's dollar
    dollar_est = np.e**(log_est) * 1000 * SCALE_FACTOR
    dollar_up = np.e**(upper) * 1000 * SCALE_FACTOR
    dollar_low = np.e**(lower) * 1000 * SCALE_FACTOR

    rounded_est = np.around(dollar_est,-3)
    rounded_up = np.around(dollar_up,-3)
    rounded_low = np.around(dollar_low,-3)

    print(f'The estimate property value is USD {rounded_est}')
    print(f'At {conf}% confidence of valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_up} at higher end')

In [54]:
get_dollar_estimate(rm=0, ptratio=200, chas=True)

That is unrealistic! Try again
